##### <p> Samuel Wolfe <br> July 26, 2023 <br> MSBA 206 <br> DMBA Case 21.4 Part 2</p>

In [328]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import requests
import io
import matplotlib.pylab as plt
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [329]:
def readFile(url):
    download = requests.get(url).content
    # Reading the downloaded content and turning it into a pandas dataframe
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    return df
def statslist(df):
    dfStats = pd.DataFrame({'Mean' : df.mean(numeric_only=True),
            'SD' : df.std(numeric_only=True),
            'Min' : df.min(),
            'Max' : df.max(),
            'Median' : df.median(numeric_only=True),
            })
    return dfStats
def categorize(df):
    for x in df:
        df[x] = df[x].astype('category')
    return df

In [330]:
dfVote = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/Voter-Persuasion.csv")
 #do not need these as they are duplicates of values or states as not useable
dropCol = ['VOTER_ID','SET_NO','Partition','opposite','MOVED_AD']
dfVote = dfVote.drop(columns=dropCol)
outcome = 'MOVED_A'

In [331]:
X = pd.get_dummies(dfVote.drop(columns=['MOVED_A','COMM_PT']), drop_first=True)
predictors = X.columns.to_list()
y = dfVote[outcome]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.4, random_state=0)

In [332]:
clf = LassoCV().fit(X, y)
importance = np.abs(clf.coef_)

In [333]:
idx_third = importance.argsort()[-3]
threshold = importance[idx_third] + 0.01

idx_features = (-importance).argsort()[:10]
name_features = np.array(predictors)[idx_features]
print('Selected features: {}'.format(name_features))

sfm = SelectFromModel(clf, threshold=threshold)
sfm.fit(X, y)
X_transform = sfm.transform(X)

n_features = sfm.transform(X).shape[1]

Selected features: ['CAND1S_S' 'ED_4COL' 'COMM_CAR' 'NH_WHITE' 'E_PELIG' 'COMM_LT10' 'AGE'
 'PP_PELIG' 'M_MAR' 'HH_ND']


In [334]:
predictors = ['CAND1S_S','ED_4COL','COMM_CAR','NH_WHITE','E_PELIG','COMM_LT10','AGE','PP_PELIG','M_MAR','HH_ND']

#### Importing dataset

In [335]:
dfVote = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/Voter-Persuasion.csv")
 #do not need these as they are duplicates of values or states as not useable
dropCol = ['VOTER_ID','SET_NO','Partition','MOVED_AD','opposite']
dfVoteDum = pd.get_dummies(dfVote.drop(columns=dropCol))
dfVoteDum = dfVoteDum[predictors]
outcome = 'MOVED_A'
classes = [0,1]

#### Binning

In [337]:
#dfVoteDum['ED_4COL'] = pd.cut(x=dfVoteDum['ED_4COL'], bins=[-1,4,20], labels=[0,1],  precision=0)
dfVoteDum['ED_4COL'] = pd.qcut(dfVoteDum['ED_4COL'], q=10, labels=[0,1,2,3,4,5,6,7,8,9],  precision=0)

In [338]:
#dfVoteDum['ED_4COL'] = pd.cut(x=dfVoteDum['ED_4COL'], bins=[-1,4,20], labels=[0,1],  precision=0)
dfVoteDum['COMM_CAR'] = pd.qcut(dfVoteDum['COMM_CAR'], q=5, labels=[0,1,2,3,4],  precision=0)

In [339]:
#dfVoteDum['ED_4COL'] = pd.cut(x=dfVoteDum['ED_4COL'], bins=[-1,4,20], labels=[0,1],  precision=0)
dfVoteDum['M_MAR'] = pd.qcut(dfVoteDum['M_MAR'], q=5, labels=[0,1,2,3,4],  precision=0)

In [340]:
#dfVoteDum['ED_4COL'] = pd.cut(x=dfVoteDum['ED_4COL'], bins=[-1,4,20], labels=[0,1],  precision=0)
dfVoteDum['AGE'] = pd.qcut(dfVoteDum['AGE'], q=3, labels=[0,1,2],  precision=0)

In [341]:
#dfVoteDum['ED_4COL'] = pd.cut(x=dfVoteDum['ED_4COL'], bins=[-1,4,20], labels=[0,1],  precision=0)
dfVoteDum['COMM_LT10'] = pd.qcut(dfVoteDum['COMM_LT10'], q=4, labels=[0,1,2,3],  precision=0)

In [342]:
#dfVoteDum['ED_4COL'] = pd.cut(x=dfVoteDum['ED_4COL'], bins=[-1,4,20], labels=[0,1],  precision=0)
dfVoteDum['E_PELIG'] = pd.qcut(dfVoteDum['E_PELIG'], q=4, labels=[0,1,2,3],  precision=0)

In [343]:
#dfVoteDum['ED_4COL'] = pd.cut(x=dfVoteDum['ED_4COL'], bins=[-1,4,20], labels=[0,1],  precision=0)
dfVoteDum['NH_WHITE'] = pd.qcut(dfVoteDum['NH_WHITE'], q=5, labels=[0,1,2,3,4],  precision=0)

#### Converting all categories into category

In [344]:
dfVoteDum = categorize(dfVoteDum)

#### Adding Partition and MOVED_A back to dfVoteDum

In [345]:
dfVoteDum['Partition'] = dfVote['Partition']
dfVoteDum['MOVED_A'] = dfVote['MOVED_A']

## 21.4.3

#### Split into partitions

In [346]:
scaler = preprocessing.StandardScaler()
scaler.fit(dfVoteDum[predictors])
X_dfVoteDum = scaler.transform(dfVoteDum.drop(columns=outcome).drop(columns='Partition'))
Y_dfVoteDum = dfVoteDum[outcome]

dfVote_T = dfVoteDum[dfVoteDum.Partition == 'T'].drop(columns='Partition')
X_train = dfVote_T.drop(columns=outcome)
Y_train = dfVote_T[outcome].to_frame()

dfVote_V = dfVoteDum[dfVoteDum.Partition == 'V'].drop(columns='Partition')
X_valid = dfVote_V.drop(columns=outcome)
Y_valid = dfVote_V[outcome].to_frame()

#### Normalizing the data

In [347]:
#scaler = preprocessing.StandardScaler()
scaler = MinMaxScaler()
scaler.fit(X_train)
# Transform the predictors of training, validation
X_train_norm = scaler.transform(X_train)
X_valid_norm = scaler.transform(X_valid)

#### Running full dataset knn ranges

In [348]:
results = []
for k in range(1,20,2):
    kfold = KFold(n_splits=10,random_state=1,shuffle=True)
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(estimator=knn,
                             X=X_dfVoteDum, y=Y_dfVoteDum, cv=kfold)
    results.append({
        'k' : k,
        'mean accuracy' : f'{scores.mean():.2%}' ,
        'standard deviation' : f'{scores.std():.2%}'
    })
results = pd.DataFrame(results).sort_values(by=['mean accuracy'],ascending=False)
print(results)

    k mean accuracy standard deviation
9  19        82.81%              1.03%
8  17        82.72%              1.04%
7  15        82.62%              1.31%
6  13        82.42%              1.21%
5  11        82.37%              1.18%
4   9        82.21%              1.31%
3   7        82.01%              1.15%
2   5        80.84%              1.37%
1   3        79.46%              1.47%
0   1        75.42%              1.17%


#### Selecting knn == 19

In [359]:
knn19 = KNeighborsClassifier(n_neighbors=19).fit(X_train_norm, Y_train.MOVED_A)
print('KNN == 19')
classificationSummary(Y_valid, knn19.predict(X_valid_norm))

KNN == 19
Confusion Matrix (Accuracy 0.8334)

       Prediction
Actual    0    1
     0 2123  448
     1  227 1254


## Naive Bayes

## Preparing data for Naive Bayes
#### verifying dtypes

In [350]:
dfVoteDum.dtypes

CAND1S_S     category
ED_4COL      category
COMM_CAR     category
NH_WHITE     category
E_PELIG      category
COMM_LT10    category
AGE          category
PP_PELIG     category
M_MAR        category
HH_ND        category
Partition      object
MOVED_A         int64
dtype: object

#### claiming new predictors

In [351]:
#predictors = dfVoteDum.drop(columns=['Partition','MOVED_A']).columns.to_list()

#### Splitting the data by T and V

In [352]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(dfVoteDum[predictors])
X_dfVoteDum = dfVoteDum.drop(columns=outcome).drop(columns='Partition')

Y_dfVoteDum = dfVoteDum[outcome]

dfVote_T = dfVoteDum[dfVoteDum.Partition == 'T'].drop(columns='Partition')
X_train = dfVote_T.drop(columns=outcome)
Y_train = dfVote_T[outcome].to_frame()

dfVote_V = dfVoteDum[dfVoteDum.Partition == 'V'].drop(columns='Partition')
X_valid = dfVote_V.drop(columns=outcome)
Y_valid = dfVote_V[outcome].to_frame()

#### Running Naive Bayes

In [353]:
vote_nb = MultinomialNB(alpha=1)
vote_nb.fit(X_train, Y_train.MOVED_A)

# predict probabilities
predProb_train = vote_nb.predict_proba(X_train)
predProb_valid = vote_nb.predict_proba(X_valid)

# predict class membership
y_train_pred = vote_nb.predict(X_train)
y_valid_pred = vote_nb.predict(X_valid)

#### Checking probability of all predictors vs outcome

In [354]:
pd.set_option('display.precision', 4)
# probability of flight status
print(dfVote_T[outcome].value_counts() / len(dfVote_T))
print()

for predictor in predictors:
    # construct the frequency table
    df = dfVote_T[[outcome, predictor]]
    freqTable = df.pivot_table(index=outcome, columns=predictor, aggfunc=len)

    # divide each row by the sum of the row to get conditional probabilities
    propTable = freqTable.apply(lambda x: x / sum(x), axis=1)
    print(propTable)
    print()
pd.reset_option('display.precision')

MOVED_A
0    0.6212
1    0.3788
Name: count, dtype: float64

CAND1S_S   False    True
MOVED_A                 
0         0.2087  0.7913
1         0.8810  0.1190

ED_4COL       0       1       2       3       4       5       6       7  \
MOVED_A                                                                   
0        0.1494  0.0647  0.0996  0.1556  0.0774  0.0774  0.1069  0.0825   
1        0.1372  0.0746  0.0706  0.1687  0.0502  0.0666  0.1305  0.0830   

ED_4COL       8       9  
MOVED_A                  
0        0.1101  0.0763  
1        0.1256  0.0932  

COMM_CAR       0       1       2       3       4
MOVED_A                                         
0         0.2663  0.1088  0.2198  0.1978  0.2073
1         0.3600  0.1061  0.1984  0.1585  0.1771

NH_WHITE       0       1       2       3       4
MOVED_A                                         
0         0.1710  0.2536  0.1608  0.2533  0.1613
1         0.2708  0.1975  0.1505  0.2326  0.1487

E_PELIG       0       1       2       

In [355]:
print('Test Data Set')
classificationSummary(Y_train, y_train_pred, class_names=classes) 
print()
print('Validation Data Set')
classificationSummary(Y_valid, y_valid_pred, class_names=classes) 

Test Data Set
Confusion Matrix (Accuracy 0.7745)

       Prediction
Actual    0    1
     0 3184  511
     1  830 1423

Validation Data Set
Confusion Matrix (Accuracy 0.7786)

       Prediction
Actual    0    1
     0 2179  392
     1  505  976


In [360]:
(0.8334-0.7786)*100

5.4800000000000075

## 24.1.4
#### Between the KNN model and the Naive Bayes method, the most accurate was KNN, by an improvement of 5.48%. While Knn appears to have the edge over Naive Bayes, I am inclined to pick Naive Bayes because with this type of dataset quick categorical results on hundreds of thousands of people means Naive Bayes may produce better results long term.

## 24.1.5

In [361]:
pd.set_option('display.width', 250)
df = pd.concat([pd.DataFrame({'actual': Y_valid.MOVED_A, 'predicted': y_valid_pred}),
                pd.DataFrame(predProb_valid, index=Y_valid.index)], axis=1)
print(df.head(3))
pd.reset_option('display.width')

   actual  predicted         0         1
0       0          0  0.901653  0.098347
8       1          1  0.389598  0.610402
9       0          0  0.901066  0.098934


#### I will admit to resorting to redoing my 24.1.1-2 analysis using a newer method. I believe this effort has been put to good use though. My Naive Bayes model is able to predict accurately the first three models.

In [363]:
pd.set_option('display.width', 250)
pd.set_option('display.precision', 4)
# probability of flight status
print(dfVote_V['MOVED_A'].value_counts() / len(dfVote_V))
print()

for predictor in predictors:
    # construct the frequency table
    df = dfVote_V[['MOVED_A', predictor]]
    freqTable = df.pivot_table(index='MOVED_A', columns=predictor, aggfunc=len)

    # divide each row by the sum of the row to get conditional probabilities
    propTable = freqTable.apply(lambda x: x / sum(x), axis=1)
    print(propTable)
    print()
pd.reset_option('display.precision')
pd.reset_option('display.width')

MOVED_A
0    0.6345
1    0.3655
Name: count, dtype: float64

CAND1S_S   False    True
MOVED_A                 
0         0.2096  0.7904
1         0.8953  0.1047

ED_4COL       0      1       2       3       4       5       6       7       8       9
MOVED_A                                                                               
0        0.1599  0.072  0.0910  0.1478  0.0615  0.0852  0.1081  0.0817  0.1109  0.0821
1        0.1404  0.077  0.0756  0.1789  0.0466  0.0722  0.1290  0.0905  0.1107  0.0790

COMM_CAR       0       1       2       3       4
MOVED_A                                         
0         0.2657  0.0941  0.2415  0.2011  0.1976
1         0.3801  0.1040  0.1783  0.1729  0.1648

NH_WHITE       0       1       2       3       4
MOVED_A                                         
0         0.1816  0.2509  0.1641  0.2419  0.1614
1         0.2924  0.1904  0.1506  0.2384  0.1283

E_PELIG      0       1       2       3
MOVED_A                               
0        0.352  0